# OpenCV Imgproc

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy
import math
%matplotlib inline

plt.rcParams['figure.figsize'] = [8,6]
plt.rcParams['figure.dpi'] = 120


def imshow(img, pos=None, title=None):
    if isinstance(img, dict):
        imshow_map(img)
        return
    if isinstance(img, list):
        imshow_list(img)
        return
    else:
        if pos != None:
            plt.subplot(pos)
        if (len(img.shape) >=3 and img.shape[2] == 3) or (len(img.shape) == 2):
            plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGBA))
        else:
            plt.imshow(img)
        if title != None:
            plt.title(title)

def imshow_list(lp):
    imshow_map(dict([(idx, value)for idx,value in enumerate(lp)]))


def imshow_map(mp):
    num = len(mp)
    col = 3 if num >= 3 else num % 3 
    row = math.ceil(num / 3)
    idx = 0
    for title in mp:
        img = mp[title]
        idx += 1
        pos = row * 100 + col * 10 + idx
        imshow(img, pos, title)

img = cv2.imread('profile.jpg')
gray = cv2.imread('profile.jpg',cv2.IMREAD_GRAYSCALE)

## 图像阈值
`cv2.threshold(src,thresh,maxval,type)`
输入图只能为单通道图像，即灰度图

| Flag | Desc |
|:- |:- |
|BINARY|大于`thresh`取`maxval` ，否则取 `0`|
|BINARY_INV| 大于阈值取`0`，否则取`maxval` |
|TRUNC|大于阈值的部分设为阈值，否则不变|
|TOZERO|大于阈值的部分不变，否则设为 0|
|TOZERO_INV|大于阈值的部分设为0，否则不变|



In [ ]:
thresh = 144
maxval = 255
_, binary = cv2.threshold(gray,thresh,maxval,cv2.THRESH_BINARY)
_, binary_inv = cv2.threshold(gray,thresh,maxval,cv2.THRESH_BINARY_INV)
_, trunc = cv2.threshold(gray,thresh,maxval,cv2.THRESH_TRUNC)
_, tozero = cv2.threshold(gray,thresh,maxval,cv2.THRESH_TOZERO)
_, tozero_inv = cv2.threshold(gray,thresh,maxval,cv2.THRESH_TOZERO_INV)
imshow(gray,231, "Origional")
imshow(binary,232, "Binary")
imshow(binary_inv,233, "Binary_inv")
imshow(trunc,234, "Trunc")
imshow(tozero,235, "Tozero")
imshow(tozero_inv,236, "Tozero_inv")


## 平滑处理
### 均值滤波 Blur
简单的平均卷积操作，用来简单的处理噪点
`cv.blur(img,size)` 其中size是一个二元组，表示卷积大小


In [ ]:
noise = cv2.imread('noise.png')
blur5 = cv2.blur(noise,(5,5))
blur50 = cv2.blur(noise,(50,50))
imshow(noise,131, "Origion")
imshow(noise,132,"Blur 5x5")
imshow(blur50, 133,"Blur 50x50")


### 方框滤波
cv2.boxFilter(img, deepth, size, normalize)
+ deepth 可输入 -1 ，意味与输出相同
+ normalize 表示是否归一化

方框滤波与均值滤波基本相同，区别就在于可以选择不进行归一化

不进行归一化，结果可能溢出（超过uint8范围），被进行截断操作

In [ ]:
box = cv2.boxFilter(noise,-1,(5,5),normalize=True)
boxWithout = cv2.boxFilter(noise, -1, (5,5), normalize=False)
imshow(noise, 131, "Origion")
imshow(box,132,"Box Filter")
imshow(boxWithout, 133, "Box Filter Without Normalize")

### 高斯滤波
高斯滤波的卷积核里的数值是满足高斯分布的，也就是说更重视中间的值

In [ ]:
gaussian = cv2.GaussianBlur(noise, (51,51), 1)
imshow(noise, 121, "Origion")
imshow(gaussian, 122, "Gaussian")

### 中值滤波
用中间值替代平滑处理后的结果

In [ ]:
median = cv2.medianBlur(noise, 15)
imshow(noise,121,"Origion")
imshow(median,122, "Median Blur")

## 平铺结果 hstack vstack
将数组水平或垂直合并 ~除了展示没啥用~

In [ ]:
hstack = numpy.hstack((noise,median))
imshow(hstack)

## 腐蚀与膨胀
### 腐蚀 erode
用小数覆盖大数，一般用来处理二值图

`cv2.erode(img, kernel_size, iterations=int)`

In [ ]:
cv_img = cv2.imread('CV.png')
erode = cv2.erode(cv_img, (30,30),iterations=4)
imshow(cv_img, 121, "Origion")
imshow(erode, 122, "Erode")

#### 迭代次数
iterations 可以指定腐蚀次数，如下

In [ ]:
circle = cv2.imread('circle.png')
imshow(circle,151, "Origion")
map = {}
for i in range(1,5):
    iter = (i-1)*50+1
    map[iter] = cv2.erode(circle, (30,30),iterations=iter)
    imshow(map[iter], 151+i, "Iteration=" + str(iter))


### 膨胀操作 dilate
用大值覆盖小值

`cv2.dilate(img, kernel_size, iterations=)`

In [ ]:
dilate = cv2.dilate(cv_img, (50,50),iterations=5)
imshow(cv_img, 121, "Origion")
imshow(dilate, 122, "Erode")

#### 迭代
同样，膨胀操作也可以迭代

In [ ]:

circle = cv2.imread('circle.png')
imshow(circle,151, "Origion")
map = {}
for i in range(1,5):
    iter = (i-1)*50+1
    map[iter] = cv2.dilate(circle, (30,30),iterations=iter)
    imshow(map[iter], 151+i, "Iteration=" + str(iter))

#### 与腐蚀操作之间组合

In [ ]:
cmp_erode_dilate = {
    'Origion': cv_img,
    'Erode' : cv_img,
    'Dilate': cv_img
}

for i in range(0,5):
    cmp_erode_dilate['Erode'] = cv2.erode(cmp_erode_dilate['Dilate'], (100,100), iterations=5)
    cmp_erode_dilate['Dilate'] = cv2.dilate(cmp_erode_dilate['Erode'], (100,100), iterations=5)

imshow(cmp_erode_dilate)

### 开运算
先腐蚀，后膨胀，是一种组合运算

开运算在 `morphologyEx` 中有实现，其 type 为 `cv2.MORPH_OPEN`

In [ ]:
opening = cv2.morphologyEx(cv_img, cv2.MORPH_OPEN, (50,50), iterations=5)
imshow({
    "Origion": cv_img,
    "Morph Open" : opening
})

### 闭运算
先膨胀，后腐蚀。

op 为 `cv2.MORPH_CLOSE`

In [ ]:
closing = cv2.morphologyEx(cv_img, cv2.MORPH_CLOSE, (50,50), iterations=5)
imshow({
    "Origion": cv_img,
    "Morph Close" : closing
})

### 梯度计算
膨胀结果-腐蚀结果

大多数时候会剩一个边框

op 为 `cv2.MORPH_GRADIENT`

In [ ]:
graditent = cv2.morphologyEx(cv_img, cv2.MORPH_GRADIENT, (50,50), iterations=5)
imshow({
    "Origion": cv_img,
    "Morph Graditent" : graditent
})

In [ ]:
graditent = cv2.morphologyEx(circle, cv2.MORPH_GRADIENT, (50,50), iterations=5)
imshow({
    "Origion": circle,
    "Morph Graditent" : graditent
})

### 礼帽

原始输入-开运算

也就剩余*被腐蚀*的部分

In [ ]:
tophat = cv2.morphologyEx(cv_img, cv2.MORPH_TOPHAT, (50,50), iterations=5)
imshow({
    "Origion": cv_img,
    "Morph Tophat" : tophat
})

### 黑帽
闭运算结果-原始输入

In [ ]:
blackhat = cv2.morphologyEx(cv_img, cv2.MORPH_BLACKHAT, (50,50), iterations=5)
imshow({
    "Origion": cv_img,
    "Morph Blackhat" : blackhat
})

## 图像梯度
### Sobel 算子

计算方式:

$G_x = \left[ \begin{matrix} -1 & 0 & +1 \\ -2 & 0 & +2 \\ -1 & 0 & +1 \end{matrix} \right] * A \text{  and  } G_y = \left[ \begin{matrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ +1 & + 2 & +1 \end{matrix} \right] * A$

使用`cv2.Sobel(src,ddepth, dx,dy ksize)`可选择的在某一方向进行计算

| Para | Desc |
|:- |:- |
| ddepth | 图像深度，`-1`表示与输入相同 |
| dx | 计算 x 方向 |
| dy | 计算 y 方向 |
| ksize | 核大小，int |

In [ ]:
circle = cv2.imread('circle.png')
sobelx = cv2.Sobel(circle, -1, 1, 0, ksize = 3)
sobely = cv2.Sobel(circle, -1, 0, 1, ksize = 3)
sobelxy = cv2.Sobel(circle, -1, 1, 1, ksize = 3)

imshow(
    {
        'Origion': circle,
        'Sobel X': sobelx,
        'Sobel Y': sobely,
        'Sobel XY': sobelxy
    }
)

从上图可以见到，由于右边缘计算结果为负值，OpenCV 进行了截断操作，计算结果变为了 0。

如果想要保留值，应该进行取绝对值操作，此时图像的深度应当能储存负值，及需要我们手动指定图像深度。

In [ ]:
sobelx = cv2.Sobel(circle, cv2.CV_64F, 1,0, ksize = 3)
sobelx_abs = cv2.convertScaleAbs(sobelx)
imshow({
    'Origion': circle,
    'Abs': sobelx_abs
})

另外同时处理 x 与 y 方向的运算意味计算斜边，而非计算 x 方向和 y 计算结果可能不预期。

如果想要计算x 和 y 方向，应该将 x 的计算结果与 y 的计算结果相加。

In [ ]:
circle = cv2.imread('circle.png')
sobelx = cv2.Sobel(circle, cv2.CV_64F, 1, 0, ksize = 3)
sobely = cv2.Sobel(circle, cv2.CV_64F, 0, 1, ksize = 3)
sobelxy = cv2.Sobel(circle, cv2.CV_64F, 1, 1, ksize = 3)

sobelx = cv2.convertScaleAbs(sobelx)
sobely = cv2.convertScaleAbs(sobely)
sobelxy = cv2.convertScaleAbs(sobelxy)
plus = cv2.add(sobelx,sobely)

imshow({
    'Sobel X': sobelx,
    'Sobel Y': sobely,
    'Sobel X and Y': sobelxy,
    'Sobel X + Sobel Y': plus,
})

对复杂图片的处理:

In [ ]:

ros = cv2.imread('profile.jpg')
sobelx = cv2.Sobel(ros, cv2.CV_64F, 1, 0, ksize = 3)
sobely = cv2.Sobel(ros, cv2.CV_64F, 0, 1, ksize = 3)
sobelxy = cv2.Sobel(ros, cv2.CV_64F, 1, 1, ksize = 3)

sobelx = cv2.convertScaleAbs(sobelx)
sobely = cv2.convertScaleAbs(sobely)
sobelxy = cv2.convertScaleAbs(sobelxy)
plus = cv2.add(sobelx,sobely)

imshow({
    'Origion': ros,
    'Sobel X': sobelx,
    'Sobel Y': sobely,
    'Sobel X and Y': sobelxy,
    'Sobel X + Sobel Y': plus,
})

### Scharr 算子
计算方式：
$G_x = \left[ \begin{matrix} -3 & 0 & 3 \\ -10 & 0 & 10 \\ -3 & 0 & 3 \end{matrix} \right] * A\text{  and  } G_y = \left[ \begin{matrix} -3 & -10 & -3 \\ 0 & 0 & 0 \\ -3 & -10 & -3 \end{matrix} \right] * A$
 
 与 Sobel 算子相比，整体计算方式相同，但是更加重视中间的内容，对结果也更敏感点
 
 **无法同时计算 x 和 y**

In [ ]:
scharrx = cv2.Scharr(circle, cv2.CV_64F,1,0)
scharry = cv2.Scharr(circle, cv2.CV_64F,0,1)
scharrx = cv2.convertScaleAbs(scharrx)
scharry = cv2.convertScaleAbs(scharry)
plus = cv2.add(scharrx,scharry)

imshow({
    'Scharr X': scharrx,
    'Scharr Y': scharry,
    'Scharr X + Scharr': plus,
})

对复杂图片的处理:

In [ ]:
ros = cv2.imread('profile.jpg')
scharrx = cv2.Scharr(ros, cv2.CV_64F,1,0)
scharry = cv2.Scharr(ros, cv2.CV_64F,0,1)
scharrx = cv2.convertScaleAbs(scharrx)
scharry = cv2.convertScaleAbs(scharry)
plus = cv2.add(scharrx,scharry)

imshow({
    'Origion': ros,
    'Scharr X': scharrx,
    'Scharr Y': scharry,
    'Scharr X + Scharr': plus,
})


### Laplacian 算子
计算方式：
$G = \left[ \begin{matrix} 0 & 1 & 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0 \end{matrix} \right] * A$

像是周围点与边缘点进行比较，像空间变化率？对变化更敏感，但对噪音点也敏感

In [ ]:

lap = cv2.Laplacian(circle, cv2.CV_64F, ksize = 3)
lap = cv2.convertScaleAbs(lap)
imshow({
    'Origion': circle,
    'Laplacian': lap
})

In [ ]:
ros = cv2.imread('profile.jpg')
ros_lap = cv2.Laplacian(ros, cv2.CV_64F, ksize = 3)
ros_lap = cv2.convertScaleAbs(ros_lap)
imshow({
    'Origion': ros,
    'Laplacian': ros_lap
})

### Canny 边缘检测

流程：

1. 使用高斯滤波器，平滑图像，消除噪声
0. 计算图像中的每个像素的梯度强度和方向
0. 应用非极大值抑制(Non-Maximum Suppression)，消除边缘检测带来的杂散影响
0. 应用双阈值(Double-Threshold) 检测，确定真实的和潜在的边缘
0. 通过抑制孤立的弱边缘完成边缘检测

#### 双阈值

当 $梯度值 > maxval$ 时，处理为边界

当 $ maxval > 梯度值 > minval$时，若有边界链接则保留，否则舍弃

当 $minval > 梯度值$，舍弃

In [ ]:
ros = cv2.imread('profile.jpg',cv2.IMREAD_GRAYSCALE)
imshow({
    'Origion': ros,
    '(50,100)': cv2.Canny(ros,50,100),
    '(100,150)': cv2.Canny(ros, 100,150),
    '(200,250)': cv2.Canny(ros, 200,250)
})

## 图像金字塔
### 高斯金字塔
#### 向下采样（缩小)
+ 将 $G_i$ 与高斯内核卷积
+ 将所有的偶数行和列去除
#### 向上采样（放大）
+ 将图像在每个方向上扩大到原来的两倍，新增的行列以 0 填充
+ 使用先前同样的内核（乘以4）与放大后的图像卷积，获得近似值

In [ ]:
import json
ros = cv2.imread('profile.jpg')
pyrup = cv2.pyrUp(ros)
pyrdown =  cv2.pyrDown(ros)
imshow({
    'Origion': ros,
    'Pyr Up': pyrup,
    'Pyr Down': pyrdown
})
print(json.dumps({
    'Origion shape': str(ros.shape),
    'Pyr up shape': str(pyrup.shape),
    'Pyr down shape': str(pyrdown.shape)}))

### Laplacian 金字塔
$L_i = G_i - PryUp(PyrDown(G_i))$

In [ ]:
def lapPry(img):
    down = cv2.pyrDown(img)
    down_up = cv2.pyrUp(down)
    return img - down_up

imshow({
    'Origion': ros,
    'Laplacian Pyr': lapPry(ros)
})

In [ ]:
imshow({
    'Origion': circle,
    'Laplacian Pyr': lapPry(circle)
})

## 轮廓检测
与边缘相比，轮廓总是闭合的

OpenCV 中的 `cv2.findContours(img, mode, method)` 中实现了轮廓检测

+ 轮廓检测模式 mode

| mode | desc |
|:- |:- |
| RETR_EXTERNAL | 仅检索外轮廓 |
| RETR_LIST | 检索所有轮廓，不建立等级关系（也就是说这个模式不存在父轮廓或者内嵌轮廓）|
| RETR_CCOMP | 检索所有轮廓，并将结果组织为两层但所有轮廓只建立两个等级关系，外围为顶层，若外围内的内围轮廓还包含了其他的轮廓信息，则内围内的所有轮廓均归属于顶层 |
| RETR_TREE | 检索所有轮廓，建立等级关系树 |

+ 轮廓逼近方式 method

| method | desc |
|:- |:- |
| CHAIN_APPROX_NONE | 以 Freeman 链码的方式输出轮廓，所有其他方式输出多边形（顶点的序列） |
| CHAIN_APPROX_SIMPLE | 压缩水平的，垂直的和斜的部分，也就是函数只保留他们终点的部分，省内存 ，但缺少细节 |

**为了提高准确性，一般使用二值图形、通常已经经过了 Canny 或 Laplacian**

返回 `contours` 和 `hierarchy`
+ `contours` 保存轮廓信息的向量，可以单独访问某个元素
+ `hierarchy` 是一个`size=4`的向量，分别表示轮廓的后、前、子、夫轮廓序号，不过不存在则为 -1

In [ ]:
shapes = cv2.imread('shape.png')
gray = cv2.cvtColor(shapes, cv2.COLOR_BGR2GRAY)
_, binary= cv2.threshold(gray, 125,255, cv2.THRESH_BINARY)
binary = cv2.Canny(binary, 125,200)

def doContour(mode):
    contours, hierarchy = cv2.findContours(binary, mode, cv2.CHAIN_APPROX_NONE)
    canvas = shapes.copy()
    draw_all = cv2.drawContours(canvas, contours, -1, (0,0,255), 5)
    return draw_all



imshow({
    'Origion': shapes,
    'Canny': binary,
    'External': doContour(cv2.RETR_EXTERNAL) ,
    'List': doContour(cv2.RETR_LIST) ,
    'CComp': doContour(cv2.RETR_CCOMP) ,
    'Tree': doContour(cv2.RETR_TREE) 
})

### 画轮廓
`drawContours(src, contours, idx, color, size)`

注意：python 中此方法会修改 src 对象

In [ ]:

shapes = cv2.imread('shape.png')
gray = cv2.cvtColor(shapes, cv2.COLOR_BGR2GRAY)
_, binary= cv2.threshold(gray, 125,255, cv2.THRESH_BINARY)
binary = cv2.Canny(binary, 125,200)

contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
canvas = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
draw_all = cv2.drawContours(canvas, contours, 1, (0,0,255), 5)
draw_all = cv2.drawContours(canvas, contours, 2, (0,0,255), 5)
imshow([shapes, draw_all])

### 轮廓特征
面积与边长
+ 面积 `cv2.contourArea(contour)`
+ 周长 `cv2.arcLength(contour, close)`

In [ ]:
idx = 3
canvas = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
imshow(cv2.drawContours(canvas, contours, idx, (0,0,255), 3))
area = cv2.contourArea(contours[idx])
arcLen = cv2.arcLength(contours[idx], True)
print("Area=\t\t{}\nArcLength=\t{}".format(area,arcLen))

### 轮廓近似
`cv2.approxPolyDP(contour, epsilon, close)`

| arg | desc |
|:- |:- |
| contour | 轮廓 |
| epsilon | 轮廓上的点到近似线上的最小距离，数值越小拟合约近似 (拟跟没拟一样)|
| closed | 是否闭合 |

In [ ]:
idx = 2
def doProcess(esp):
    canvas = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
    approx = 0
    if esp == -1:
        approx = contours[idx]
    else:
        approx = cv2.approxPolyDP(contours[idx], esp, True)
    print(len(approx))
    return cv2.drawContours(canvas, [approx], 0, (0,0,255), 3)
    

arcLen = cv2.arcLength(contours[idx], True)

imshow(
    {
        'Origion': doProcess(-1),
        '0.01': doProcess(0.01 * arcLen),
        '0.02': doProcess(0.02 * arcLen),
        '0.035': doProcess(0.035 * arcLen),
        '0.05': doProcess(0.05 * arcLen),
        '0.1': doProcess(0.1 * arcLen),
        '0.2': doProcess(0.2 * arcLen),
    }
)


### 外接

+ 外接矩形： `cv2.boundRect(contour)`
+ 外接圆： `cv2.minEnclosingCircle(contour)`

In [ ]:
circle = cv2.imread('circle.png')

canny = cv2.Canny(circle, 125, 200)
contours, _  = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

drawContour = circle.copy()
cv2.drawContours(drawContour, contours, 0, (0,0,255),3)

x,y,w,h = cv2.boundingRect(contours[0])
drawRect = circle.copy()
cv2.rectangle(drawRect, (x,y), (x+w, y + h), (0,0,255), 3)

(x,y), r = cv2.minEnclosingCircle(contours[0])
drawCir = circle.copy()
cv2.circle(drawCir, (int(x),int(y)), int(r), (0,0,255), 3)

imshow(
    {
        'Origion': drawContour,
        'Bounding Rect': drawRect,
        'Circle': drawCir
    }
)


## 统计直方图
OpenCV 中可以统计 [0,255] 内的颜色直方图，使用 `cv2.calcHist(image,channels,maks,histSize,size)`

| arg | desc |
|:- |:- |
| image | 原图应为 `uint8` 或者 `float32` |
| channels | 统计通道？ `[0]` 标记灰度图，如果是 BGR 从 `[0][1][2]` 中选择一个 |
| mask | 遮罩图像，可以为 `None`。 如果想要统计位置范围内的颜色的直方图应制作对应该遮罩 |
| histSize | BIN 数量，传数组 |
| size | 像素范围，一般为 `[0,256]` |

In [ ]:
img = cv2.imread('noise.png', cv2.IMREAD_GRAYSCALE)
# hist = cv2.calcHist([img], [0], None, [256], [0,256])
plt.hist(img.ravel(),256)
plt.show()
None

### 彩色图

In [ ]:
img = cv2.imread('noise.png')
color = ('b', 'g', 'r')
for i, col in enumerate(color):
    histr = cv2.calcHist([img], [i], None, [256], [0, 256])
    plt.plot(histr, color = col)
    plt.xlim([0,256])